## module

In [2]:
import json
from langchain.document_loaders import JSONLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter


import os
# JSON 파일 경로 설정
import json
import time
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### JSON File

#### Json_Split(지식검색대화)

In [3]:
import json
import time
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os
from glob import glob

# JSON 파일 경로 설정
json_directory = "./dataset/common_senses"
common_senses=["TL_배움과 학문"]
json_file_paths = []
for common_sense in common_senses:
    json_file_paths.extend(glob(os.path.join(json_directory, common_sense, '*.json')))

# JSON 파일을 로드하여 데이터 추출
def load_json_files(json_file_paths):
    json_datas = []
    total_files = len(json_file_paths)
    total_time = 0

    # 파일별로 진행 상황을 모니터링하기 위해 tqdm 사용
    for i, json_file_path in enumerate(tqdm(json_file_paths, desc="Loading JSON files", unit="file")):
        start_time = time.time()

        with open(json_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            if isinstance(data, list):  # 데이터가 리스트인 경우 확장
                json_datas.extend(data)
            else:  # 데이터가 리스트가 아닌 경우 추가
                json_datas.append(data)

        end_time = time.time()
        elapsed_time = end_time - start_time
        total_time += elapsed_time
        print(f"File {i+1}/{total_files} processed in {elapsed_time:.2f} seconds")

    return json_datas, total_time

# JSON 파일 로드
json_datas, total_time = load_json_files(json_file_paths)

# 총 소요 시간 출력
print(f"Total time taken for loading JSON files: {total_time:.2f} seconds")
##print(len(json_datas))  # 데이터 길이 출력

# 일부 데이터 출력
#print(json_datas[:5])


Loading JSON files:   0%|          | 0/1394 [00:00<?, ?file/s]

Loading JSON files:  18%|█▊        | 252/1394 [00:00<00:00, 2513.43file/s]

File 1/1394 processed in 0.00 seconds
File 2/1394 processed in 0.00 seconds
File 3/1394 processed in 0.00 seconds
File 4/1394 processed in 0.00 seconds
File 5/1394 processed in 0.00 seconds
File 6/1394 processed in 0.00 seconds
File 7/1394 processed in 0.00 seconds
File 8/1394 processed in 0.00 seconds
File 9/1394 processed in 0.00 seconds
File 10/1394 processed in 0.00 seconds
File 11/1394 processed in 0.00 seconds
File 12/1394 processed in 0.00 seconds
File 13/1394 processed in 0.00 seconds
File 14/1394 processed in 0.00 seconds
File 15/1394 processed in 0.00 seconds
File 16/1394 processed in 0.00 seconds
File 17/1394 processed in 0.00 seconds
File 18/1394 processed in 0.00 seconds
File 19/1394 processed in 0.00 seconds
File 20/1394 processed in 0.00 seconds
File 21/1394 processed in 0.00 seconds
File 22/1394 processed in 0.00 seconds
File 23/1394 processed in 0.00 seconds
File 24/1394 processed in 0.00 seconds
File 25/1394 processed in 0.00 seconds
File 26/1394 processed in 0.00 sec

Loading JSON files:  92%|█████████▏| 1283/1394 [00:00<00:00, 3271.06file/s]

File 528/1394 processed in 0.05 seconds
File 529/1394 processed in 0.00 seconds
File 530/1394 processed in 0.00 seconds
File 531/1394 processed in 0.00 seconds
File 532/1394 processed in 0.00 seconds
File 533/1394 processed in 0.00 seconds
File 534/1394 processed in 0.00 seconds
File 535/1394 processed in 0.00 seconds
File 536/1394 processed in 0.00 seconds
File 537/1394 processed in 0.00 seconds
File 538/1394 processed in 0.00 seconds
File 539/1394 processed in 0.00 seconds
File 540/1394 processed in 0.00 seconds
File 541/1394 processed in 0.00 seconds
File 542/1394 processed in 0.00 seconds
File 543/1394 processed in 0.00 seconds
File 544/1394 processed in 0.00 seconds
File 545/1394 processed in 0.00 seconds
File 546/1394 processed in 0.00 seconds
File 547/1394 processed in 0.00 seconds
File 548/1394 processed in 0.00 seconds
File 549/1394 processed in 0.00 seconds
File 550/1394 processed in 0.00 seconds
File 551/1394 processed in 0.00 seconds
File 552/1394 processed in 0.00 seconds


Loading JSON files: 100%|██████████| 1394/1394 [00:00<00:00, 3062.46file/s]

File 1284/1394 processed in 0.00 seconds
File 1285/1394 processed in 0.00 seconds
File 1286/1394 processed in 0.00 seconds
File 1287/1394 processed in 0.00 seconds
File 1288/1394 processed in 0.00 seconds
File 1289/1394 processed in 0.00 seconds
File 1290/1394 processed in 0.00 seconds
File 1291/1394 processed in 0.00 seconds
File 1292/1394 processed in 0.00 seconds
File 1293/1394 processed in 0.00 seconds
File 1294/1394 processed in 0.00 seconds
File 1295/1394 processed in 0.00 seconds
File 1296/1394 processed in 0.00 seconds
File 1297/1394 processed in 0.00 seconds
File 1298/1394 processed in 0.00 seconds
File 1299/1394 processed in 0.00 seconds
File 1300/1394 processed in 0.00 seconds
File 1301/1394 processed in 0.00 seconds
File 1302/1394 processed in 0.00 seconds
File 1303/1394 processed in 0.00 seconds
File 1304/1394 processed in 0.00 seconds
File 1305/1394 processed in 0.00 seconds
File 1306/1394 processed in 0.00 seconds
File 1307/1394 processed in 0.00 seconds
File 1308/1394 p

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document

import re
# 텍스트 전처리
# 문서 생성
documents = [{"text": json.dumps(item)} for item in json_datas]


In [5]:
def split_document(doc):
    return text_splitter.split_text(doc["text"])

# 문서 나누기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = []

start_time = time.time()
with ThreadPoolExecutor() as executor:
    for split in tqdm(executor.map(split_document, documents), total=len(documents), desc="Splitting documents"):
        split_docs.extend(split)

end_time = time.time()
total_time = end_time - start_time
print(f"Total time taken for splitting: {total_time:.2f} seconds")
 
# Document 객체로 변환
chunks = [Document(page_content=doc) for doc in split_docs]

max_len = 15000
for idx, doc in enumerate(chunks):
    # 길이 조정
    if len(doc.page_content) > max_len:
      print(idx)
      doc.page_content = doc.page_content[:max_len]

Splitting documents: 100%|██████████| 1394/1394 [00:00<00:00, 414612.10it/s]


Total time taken for splitting: 0.93 seconds


In [6]:
print(chunks[:1])

[Document(page_content='{"info": {"category": "\지\식\검\색 \대\화", "evaluation": {"speaker1UttAvg_rating": 4.9, "speaker1UttGrade": "\우\수", "speaker2UttAvg_rating": 4.9, "speaker2UttGrade": "\우\수"}, "id": "BK22002540", "name": "2022 \한\국\어 \블\렌\더\봇 \데\이\터 BK22002540", "topic": "\배\움\과 \학\문", "votes": [{"speaker1UttScore": 5.0, "speaker2UttScore": 4.8, "voter_id": "bk_mc_voter0016"}, {"speaker1UttScore": 5.0,')]


#### Embedding

In [7]:
from tqdm import tqdm
import time

# 아래 코드를 실행하기 전에 필요한 라이브러리를 import합니다.

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()


In [54]:

print("데이터 업로드 중...")
# 데이터 총 양
total_chunks = len(chunks)



데이터 업로드 중...


In [13]:
SYS_PROMPT = f"""
    너는 배움과 학문에 관한 질문에 답해줄 수 있는 상담 챗봇이야.
    사용자의 입력에 맞는 적절한 답변을 배움과 학문 DB를 참고하여 진행해줘.
    
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
"""

OUTPUT_FORMAT="""
### 사용자 님이 질문해주신 ...에 대한 답변은 이렇습니다!.
- ...
- ...
- ...

-
"""

#### Model

In [14]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''
3. 사용자가 입력한 답변={question} \\
4. 배움과 학문 DB={context}\\

출력은 앞서 입력된 내용을 바탕으로 출력해야합니다. 특히 분석 결과는 3,4번 항목을 참고하여 충분히 설명해야만 합니다.
출력 예시:
'''+OUTPUT_FORMAT
# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '

'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  f"""
요즘 핫한 AI기술에 대해서 알려줘
rag와 llm은 뭐야?

""" +INPUT_PROMPT # 추가적인 입력 프롬프트가 이어집니다.

response=rag_chain.invoke(input_message)
# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(response)


> ### 사용자 님이 질문해주신 요즘 핫한 AI기술에 대한 답변은 이렇습니다!
> 
> - **AI 기술의 최신 동향**: 최근 인공지능(AI) 기술은 다양한 분야에서 급속도로 발전하고 있습니다. 특히 자연어 처리(NLP), 컴퓨터 비전, 자율주행, 의료 진단 등에서 큰 성과를 보이고 있습니다. 이러한 기술들은 머신러닝(ML)과 딥러닝(DL) 알고리즘을 기반으로 하여 데이터를 분석하고 예측하는 데 사용됩니다.
> 
> - **RAG (Retrieval-Augmented Generation)**: RAG는 정보 검색과 생성 모델을 결합한 기술입니다. 이 모델은 먼저 대규모 데이터베이스에서 관련 정보를 검색한 후, 이를 바탕으로 자연어 생성 모델이 응답을 생성합니다. 이를 통해 더 정확하고 풍부한 정보를 제공할 수 있습니다. 예를 들어, 질문에 대한 답변을 생성할 때, RAG는 관련 문서를 검색하고 이를 바탕으로 답변을 생성합니다.
> 
> - **LLM (Large Language Model)**: LLM은 대규모 언어 모델을 의미합니다. 대표적인 예로는 OpenAI의 GPT-3가 있습니다. 이러한 모델은 수십억 개의 매개변수를 가지고 있으며, 방대한 양의 텍스트 데이터를 학습하여 자연어 이해와 생성 능력을 갖추고 있습니다. LLM은 텍스트 생성, 번역, 요약, 질문 응답 등 다양한 NLP 작업에 활용됩니다.
> 
> 이와 같은 AI 기술들은 앞으로도 계속 발전할 것이며, 우리의 일상생활과 다양한 산업 분야에 큰 영향을 미칠 것입니다. 더 자세한 정보는 관련 기사나 논문을 참고하시기 바랍니다.

## 안쓰는 코드

#### Text_Split

In [ ]:
import os
from glob import glob
import time
from concurrent.futures import ThreadPoolExecutor
import logging
from tqdm import tqdm
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from openai import RateLimitError

# txt 파일 경로 설정
train_med_txt_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/01.원천데이터/TS_01.의료'
train_leg_txt_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/01.원천데이터/TS_02.법률'
txt_file_paths = [train_med_txt_directory, train_leg_txt_directory]

# PDF 및 텍스트 파일들을 로드하여 Document 객체로 변환
def load_documents(txt_file_paths):
    documents = []

    for path in txt_file_paths:
        text_files = glob(os.path.join(path, '*.txt'))
        for text_file in text_files:
            with open(text_file, 'r', encoding='utf-8') as file:
                text = file.read()
                # 텍스트를 Document 객체로 변환
                text_document = Document(page_content=text, metadata={"source": text_file})
                documents.append(text_document)

    return documents

# Document 객체를 로드
document_objs = load_documents(txt_file_paths)

# 텍스트를 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = chunk_splitter.split_documents(document_objs)

# 길이 제한 설정
max_len = 15000

# 문서 길이 제한 적용
for idx, doc in enumerate(split_docs):
    if len(doc.page_content) > max_len:
        print(idx)
        split_docs[idx].page_content = doc.page_content[:max_len]

In [ ]:
import json
import time
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# JSON 파일 경로 설정
train_med_json_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/02.라벨링데이터/Training_medical.json'
train_leg_json_directory = './dataset/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/02.라벨링데이터/Training_legal.json'
json_file_paths = [train_med_json_directory, train_leg_json_directory]

# 총 예상 소요시간과 현재 진행 퍼센트를 알 수 있게 하는 함수
def load_json_files(file_paths):
    json_datas = []
    total_files = len(file_paths)
    total_time = 0

    # 파일별로 진행 상황을 모니터링하기 위해 tqdm 사용
    for i, json_file_path in enumerate(tqdm(file_paths, desc="Loading JSON files", unit="file")):
        start_time = time.time()

        with open(json_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            json_datas.extend(data['data'])

        end_time = time.time()
        elapsed_time = end_time - start_time
        total_time += elapsed_time
        print(f"File {i+1}/{total_files} processed in {elapsed_time:.2f} seconds")

    return json_datas, total_time

# JSON 파일 로드
json_datas, total_time = load_json_files(json_file_paths)

# 총 소요 시간 출력
print(f"Total time taken for loading JSON files: {total_time:.2f} seconds")

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# 문서 생성
documents = [{"text": item.get("text", "")} for item in json_datas]

# 문서 분할 함수
def split_document(doc):
    return text_splitter.split_text(doc["text"])

# 문서 분할
split_docs = []
start_time = time.time()
with ThreadPoolExecutor() as executor:
    for split in tqdm(executor.map(split_document, documents), total=len(documents), desc="Splitting documents"):
        split_docs.extend(split)
end_time = time.time()
total_time = end_time - start_time
print(f"Total time taken for splitting: {total_time:.2f} seconds")


#### Prompt

In [ ]:
SYS_PROMPT = f"""
    기사 정보에 대한 값을 도출하고 시스템 프롬프트 설정을 하는 시스템입니다.\\
    정보를 기준으로 답변해주세요. \\
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
"""

#### Embedding

In [ ]:
from tqdm import tqdm

# 시작 시간 기록
start_time = time.time()

# ChromaDB에 분할된 텍스트 저장
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    ids=[str(i) for i in range(len(chunks))]  # 각 문서에 대한 고유한 ID 생성
)

# 종료 시간 기록
end_time = time.time()

# 총 소요 시간 계산
total_time = end_time - start_time

# 총 소요 시간 출력
print(f"Total time taken for creating ChromaDB: {total_time:.2f} seconds")


In [ ]:

# Retriever 생성
retriever = vectordb.as_retriever()

# 예시 질문에 대한 답변 생성
query = "인공지능융합공학부에 대해서 알려줘"
results = retriever.retrieve(query)
print(results)

#### Model

In [ ]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = SYS_PROMPT + '''
사용자의 질문={question} \\
참고할 document={context}\\

출력은 앞서 입력된 내용을 바탕으로 출력해야합니다. 
출력 예시:
'''



# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '

'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  f"""
Q: KGLS 답변 결과와 인적사항을 입력해주세요! \\
A: 네 알겠습니다! 양식은 다음과 같습니다
    KGLS data format\\
    
""" + INPUT_PROMPT  # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
output=rag_chain.invoke(input_message)
to_markdown(output)
